#### Step 1: Import gurobipy module

In [1]:
import gurobipy as gp
from gurobipy import GRB

#### Step 2: Define your model

In [2]:
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-08


#### Step 4: Define your parameters

In [3]:
import random
import math

random.seed(10)
#Number of nodes in the network
N = 18

#
D = 18

#cost
c = random.randrange(3,5)

#penalty cost increase
p = random.uniform(1.5, 2.0)

#discount time limit
H = random.randrange(20,30)

#Big M
M = 10**15

t = {}

#tij time between nodes
arcs = [(1,2), (1,3), 
        (2,4), (2,5), (2, 6), 
        (3, 6), 
        (4, 5), (4, 9), 
        (5,8),
        (6, 7),(6, 8),
        (7, 8),(7, 11),
        (8, 9),(8, 10),(8, 11),
        (9, 12),(9, 13),
        (10,12),
        (11,12),(11,14),
        (12,13),(12,14),
        (13,15),(14,16),
        (14,17),
        (15,16),(15,18),
        (16,17),(16,18),
        (17,18)]

for i,j in arcs:
    t[i,j] = random.randrange(5,10)  

print(t)
print(D)
#dj the delay at each node
d = []
for i in range(N):
    number = random.randrange(0,3)  
    d.append(number)
d[D - 1] = 0
d[0] = 0

print(d)




{(1, 2): 5, (1, 3): 6, (2, 4): 8, (2, 5): 8, (2, 6): 7, (3, 6): 6, (4, 5): 5, (4, 9): 9, (5, 8): 8, (6, 7): 7, (6, 8): 5, (7, 8): 6, (7, 11): 7, (8, 9): 5, (8, 10): 8, (8, 11): 6, (9, 12): 9, (9, 13): 7, (10, 12): 8, (11, 12): 8, (11, 14): 7, (12, 13): 7, (12, 14): 8, (13, 15): 6, (14, 16): 7, (14, 17): 7, (15, 16): 6, (15, 18): 8, (16, 17): 6, (16, 18): 8, (17, 18): 9}
18
[0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 0, 1, 2, 2, 1, 1, 1, 0]


#### Step 3: Define your sets

In [4]:
#number of Nodes
S = N

#### Step 5: Define your decision variables

In [5]:
# objective variables
tplus = m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, obj= 2, name = "t+")
cplus = m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, obj = 5, name = "c+")

# normal variables
tminus = m.addVar(vtype =GRB.CONTINUOUS, lb = 0.0, name = "t-")
cminus = m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, name = "c-")

x = {}
for i in range(S + 1):
    for j in range(S + 1):
        if (i, j) in t:
            x[i,j] = m.addVar(vtype=GRB.BINARY, lb=0.0, name="x_"+str(i)+str(j))

R = m.addVar(vtype=GRB.BINARY)

C = m.addVar(vtype = GRB.INTEGER, lb = 0.0)

Y = m.addVar(vtype = GRB.INTEGER, lb = 0.0)


#### Step 6: Set your objective function

In [6]:
m.modelSense = GRB.MINIMIZE

#### Step 6: Add your constraints

In [7]:
m.addConstr(Y == (sum((x[i,j] * (t[i,j] + d[j - 1])) for i,j in arcs ))) #1

m.addConstr(M*R >= Y-H) #2 

m.addConstr(C >= (c + R*c*p*(Y-H))) #3

m.addConstr((C - (cplus - cminus)) == 10) #4 

m.addConstr(sum(x[1,j] for j in range(2,S + 1) if (1,j) in arcs) == 1) #5

m.addConstr((sum(x[i,D] for i in range(S) if ((i,D) in x and i != D))) == 1) #6

for k in range(2, S):
    m.addConstr((sum(x[i,k] for i in range(S + 1) if (i,k) in x and i != k)) == (sum(x[k,j] for j in range(S + 1) if (k,j) in x and j != k))) #7

m.addConstr((Y - (tplus - tminus)) == 20) #8


m.update()

#### Step 7: Solve the model

In [8]:
#m.Params.TimeLimit=300 #(seconds) optional: sets a time limit for optimization only if you need to prematurely stop the solution procedure
m.optimize()

print(Y)
print(R)
if m.status == GRB.Status.OPTIMAL:
    print('The final solution is:')
    for i,j in arcs:
        if(x[i,j].x > 0):
            print(i, j, x[i,j].x)
            print(t[i,j])
            print(d[j-1])




Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 22 rows, 38 columns and 102 nonzeros
Model fingerprint: 0xf9ed695a
Model has 1 quadratic constraint
Variable types: 4 continuous, 34 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+15]
  QMatrix range    [5e+00, 5e+00]
  QLMatrix range   [1e+00, 1e+02]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
  QRHS range       [3e+00, 3e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 6 rows and 9 columns
Presolve time: 0.00s
Presolved: 16 rows, 29 columns, 74 nonzeros
Variable types: 0 continuous, 29 integer (26 binary)
Found heuristic solution: objective 511.0000000

Root relaxation: objective 5.096167e+02, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current 

#### Step 8: Print variable values  (The Messy Way)

In [9]:
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))
    
#https://docs.python.org/2.4/lib/typesseq-strings.html

t+ 28
c+ 91
t- 0
c- -0
x_12 1
x_13 0
x_24 1
x_25 -0
x_26 -0
x_36 -0
x_45 -0
x_49 1
x_58 -0
x_67 -0
x_68 -0
x_78 -0
x_711 -0
x_89 -0
x_810 -0
x_811 -0
x_912 -0
x_913 1
x_1012 0
x_1112 -0
x_1114 -0
x_1213 -0
x_1214 -0
x_1315 1
x_1416 -0
x_1417 -0
x_1516 -0
x_1518 1
x_1617 -0
x_1618 -0
x_1718 0
C35 1
C36 101
C37 48


#### Step 8 Alternate: Print the solution (The Easy To Read Way)

In [10]:
print('\nTOTAL COSTS: %g' % m.objVal) #gets the objective function value
print('SOLUTION:')
for i in range(I):
    if y[i].x > 0.99:   #chooses the binary y variables which get the value of 1
        print('Facility %s open' % i)
        for j in range(J):
            if x[i, j].x > 0:  #chooses the continuous x variables which get a nonzero value
                print('  ships %g thousand units to customer %s' %
                      (x[i,j].x, j))
    else:
        print('Facility %s closed!' % i)


TOTAL COSTS: 511
SOLUTION:


NameError: name 'I' is not defined

#### Optional Step: Exporting the LP File for Debugging
Do you think that something is wrong with your model? Export it to an LP file and check if it is in the desired shape.

In [ ]:
m.write("checkModel.lp")